In [1]:
#create a spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").\
                                     appName("spark_on_docker").\
                                     getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/02 01:11:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df = spark.read.format("csv")\
.option("header", "true")\
.option("inferSchema", "true")\
.load("work/TheDefinitiveGuide/Spark-The-Definitive-Guide/data/retail-data/by-day/2010-12-01.csv")
df.printSchema()
df.createOrReplaceTempView("dfTable")


root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



In [43]:
from pyspark.sql.functions import current_date, current_timestamp
dateDF = spark.range(10)\
.withColumn("today", current_date())\
.withColumn("now", current_timestamp())\
.withColumn("tomorrow", current_date() + 1)
dateDF.createOrReplaceTempView("dateTable")
dateDF.printSchema()

root
 |-- id: long (nullable = false)
 |-- today: date (nullable = false)
 |-- now: timestamp (nullable = false)
 |-- tomorrow: date (nullable = false)



In [46]:
spark.sql("select * from dateTable").show(1)

+---+----------+--------------------+----------+
| id|     today|                 now|  tomorrow|
+---+----------+--------------------+----------+
|  0|2021-12-02|2021-12-02 02:17:...|2021-12-03|
+---+----------+--------------------+----------+
only showing top 1 row



In [48]:
from pyspark.sql.functions import date_add, date_sub
dateDF.select("today", date_sub(col("today"), 5), date_add(col("today"), 5)).show(1)

+----------+------------------+------------------+
|     today|date_sub(today, 5)|date_add(today, 5)|
+----------+------------------+------------------+
|2021-12-02|        2021-11-27|        2021-12-07|
+----------+------------------+------------------+
only showing top 1 row



In [49]:
from pyspark.sql.functions import datediff, months_between, to_date
dateDF.withColumn("week_ago", date_sub(col("today"), 7))\
.select(datediff(col("week_ago"), col("today"))).show(1)

dateDF.select(
to_date(lit("2016-01-01")).alias("start"),
to_date(lit("2017-05-22")).alias("end"))\
.select(months_between(col("start"), col("end"))).show(1)


+-------------------------+
|datediff(week_ago, today)|
+-------------------------+
|                       -7|
+-------------------------+
only showing top 1 row

+--------------------------------+
|months_between(start, end, true)|
+--------------------------------+
|                    -16.67741935|
+--------------------------------+
only showing top 1 row



In [53]:
spark.sql("SELECT to_date('2016-01-01'), months_between('2016-01-01', '2017-05-22'), \
datediff('2016-01-01', '2017-05-22') \
FROM dateTable").show(1)

+-------------------+--------------------------------------------+--------------------------------+
|to_date(2016-01-01)|months_between(2016-01-01, 2017-05-22, true)|datediff(2016-01-01, 2017-05-22)|
+-------------------+--------------------------------------------+--------------------------------+
|         2016-01-01|                                -16.67741935|                            -507|
+-------------------+--------------------------------------------+--------------------------------+
only showing top 1 row



to_date function allows you to convert a string to a date, optionally with a specified format.

In [54]:
from pyspark.sql.functions import to_date, lit
spark.range(5).withColumn("date", lit("2017-01-01"))\
.select(to_date(col("date"))).show(1)

+-------------+
|to_date(date)|
+-------------+
|   2017-01-01|
+-------------+
only showing top 1 row



In [55]:
dateDF.select(to_date(lit("2016-20-12")),to_date(lit("2017-12-11"))).show(1)

+-------------------+-------------------+
|to_date(2016-20-12)|to_date(2017-12-11)|
+-------------------+-------------------+
|               null|         2017-12-11|
+-------------------+-------------------+
only showing top 1 row



In [79]:
from pyspark.sql.functions import to_date
dateFormat = "yyyy-dd-MM"
cleanDateDF = spark.range(1).select(
to_date(lit("2017-12-11"), dateFormat).alias("date"),
to_date(lit("2017-20-12"), dateFormat).alias("date2"))
cleanDateDF.createOrReplaceTempView("dateTable2")

In [80]:
cleanDateDF.show(1)

+----------+----------+
|      date|     date2|
+----------+----------+
|2017-11-12|2017-12-20|
+----------+----------+



In [81]:
spark.sql("SELECT to_date(date, 'yyyy-dd-MM'), to_date(date2, 'yyyy-dd-MM'), to_date(date) FROM dateTable2").show(1)

+-------------------------+--------------------------+-------------+
|to_date(date, yyyy-dd-MM)|to_date(date2, yyyy-dd-MM)|to_date(date)|
+-------------------------+--------------------------+-------------+
|               2017-11-12|                2017-12-20|   2017-11-12|
+-------------------------+--------------------------+-------------+



In [82]:
from pyspark.sql.functions import to_timestamp
cleanDateDF.select(to_timestamp(col("date"), dateFormat)).show()

+------------------------------+
|to_timestamp(date, yyyy-dd-MM)|
+------------------------------+
|           2017-11-12 00:00:00|
+------------------------------+



In [83]:
spark.sql("SELECT to_timestamp(date, 'yyyy-dd-MM'), to_timestamp(date2, 'yyyy-dd-MM') FROM dateTable2").show(1)

+------------------------------+-------------------------------+
|to_timestamp(date, yyyy-dd-MM)|to_timestamp(date2, yyyy-dd-MM)|
+------------------------------+-------------------------------+
|           2017-11-12 00:00:00|            2017-12-20 00:00:00|
+------------------------------+-------------------------------+



In [84]:
cleanDateDF.filter(col("date2") > lit("2017-12-12")).show()

+----------+----------+
|      date|     date2|
+----------+----------+
|2017-11-12|2017-12-20|
+----------+----------+



In [86]:
cleanDateDF.filter(col("date2") > "2017-12-12").show()

+----------+----------+
|      date|     date2|
+----------+----------+
|2017-11-12|2017-12-20|
+----------+----------+

